# Imports



In [76]:
from pathlib import Path
import numpy as np
import trimesh
from monai.data import Dataset
from monai.config import print_config
from transforms_templates.utils.utils import watermark, wide_notebook
from transforms_templates.utils.log import log

# autoreload python modules on the fly when its source is changed
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
print_config()

_ = watermark(packages=['python', 'virtualenv', 'nvidia', 'cudnn', 'hostname', 'torch', 'trimesh', 'transforms_templates'])
wide_notebook()

MONAI version: 0.3.0
Python version: 3.7.5 (default, Apr 19 2020, 20:18:17)  [GCC 9.2.1 20191008]
OS version: Linux (5.3.0-64-generic)
Numpy version: 1.19.2
Pytorch version: 1.4.0
MONAI flags: HAS_EXT = True, USE_COMPILED = False

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 3.1.1
scikit-image version: 0.17.2
Pillow version: 7.0.0
Tensorboard version: 2.4.0a20201021
gdown version: 3.12.2
TorchVision version: 0.5.0
ITK version: 5.1.0
tqdm version: 4.50.2

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies

virtualenv:     (transforms_templates) 
python:         3.7.5
hostname:       GA-970A-UD3
nvidia driver:  b'435.21'
torch:          1.4.0
trimesh:        3.8.11
transforms_templates: 


#  Data dir

In [30]:
# Setup data directory

DIR_DATA = Path("/media/Linux_4Tb/synth3D/tablets_30")
assert DIR_DATA.exists()

# Data set

In [56]:
#d = (DIR_DATA / 'corrupted')
train_ply_files = sorted((DIR_DATA / 'corrupted').glob('*.ply'))

def get_corresponding_png_file(fn, foreshortening='top', kind='rgb'):
    assert foreshortening in ['top', 'left', 'right']
    assert kind in ['rgb', 'depth']
    base = fn.with_suffix('').name
    return fn.parent.parent / 'renders' / f'{base}_{foreshortening}_{kind}.png'

def get_corresponding_meta_info_file(fn, foreshortening='top'):
    assert foreshortening in ['top', 'left', 'right']
    base = fn.with_suffix('').name
    return fn.parent.parent / 'renders' / f'{base}_{foreshortening}_meta_info.json'

def get_files_dict(fn):
    {'mesh': fn, 'image2d': get_corresponding_png_file(fn), 'image2d_meta_dict': get_corresponding_meta_info_file(fn)}
    return 

data_dicts = [get_files_dict(fn) for fn in train_ply_files]

In [61]:
ds = Dataset(data=data_dicts)

In [62]:
ds[0]

{'mesh': PosixPath('/media/Linux_4Tb/synth3D/tablets_30/corrupted/tablets_30_00000.ply'),
 'image2d': PosixPath('/media/Linux_4Tb/synth3D/tablets_30/renders/tablets_30_00000_top_rgb.png'),
 'image2d_meta_dict': PosixPath('/media/Linux_4Tb/synth3D/tablets_30/renders/tablets_30_00000_top_meta_info.json')}

In [63]:
ds[0]['mesh'].exists(), ds[0]['image2d'].exists(), ds[0]['image2d_meta_dict'].exists()

(True, True, True)

#  Loader as transform

In [64]:
from transforms_templates.transforms.io.mesh import LoadPLYd
from transforms_templates.transforms.io.image2d import LoadPNGandJSONd
from transforms_templates.transforms.compose import Compose

#from monai.transforms import LoadDatad, LoadPNG
#from monai.config import KeysCollection
#from monai.transforms import Transform



## Test loader transform

In [68]:
load_plyd = LoadPLYd(keys=['mesh'])
load_pngd = LoadPNGandJSONd(keys=['image2d'], overwriting=True)

tfms = Compose([
    load_plyd,
    load_pngd,
])

In [69]:
# object before transformation
ds[0]

{'mesh': PosixPath('/media/Linux_4Tb/synth3D/tablets_30/corrupted/tablets_30_00000.ply'),
 'image2d': PosixPath('/media/Linux_4Tb/synth3D/tablets_30/renders/tablets_30_00000_top_rgb.png'),
 'image2d_meta_dict': PosixPath('/media/Linux_4Tb/synth3D/tablets_30/renders/tablets_30_00000_top_meta_info.json')}

In [70]:
#o = load_pngd(load_plyd(ds[0]))
o = tfms(ds[0])


In [71]:
o.keys()

dict_keys(['mesh', 'image2d', 'image2d_meta_dict', 'mesh_meta_dict'])

In [72]:
log(o)

   mesh          <trimesh.Trimesh(vertices.shape=(20672, 3), faces.shape=(41340, 3))>
   image2d       shape: (256, 256, 4)   dtype: float32        min:   48.00000,  max:  255.00000,  mean:  186.18477
image2d_meta_dict:
   filename_or_obj/media/Linux_4Tb/synth3D/tablets_30/renders/tablets_30_00000_top_rgb.png
   spatial_shape (256, 256)
   format        'PNG'
   mode          'RGBA'
   width         256
   height        256
info:
   Software      'Matplotlib version3.3.2, https://matplotlib.org/'
   dpi           (100, 100)
camera_info:
   projection_matrix[[3.7290582683229117, 0.0, 0.0, 0.0], [0.0, 3.7290582683229117, 0.0, 0.0], [0.0, 0.0, -1.0, -0.1], [0.0, 0.0, -1.0, 0.0]]
   pos           [[0.4740219150535512, 0.0, 0.0, -0.01960298782994421], [0.0, 0.3884119482415088, 0.0, 0.07782934662503353], [0.0, 0.0, 0.21290340269744243, 2.037916436782144], [0.0, 0.0, 0.0, 1.0]]
   mesh_transform[[0.029978259332472904, -0.000982396127965389, 0.0005821213299304508, 0.0], [0.0009825811245980187,

In [74]:
type(o['image2d'])

numpy.ndarray

In [77]:
isinstance(o['image2d'], np.ndarray) and 

True

In [15]:
# object after transformation
o

{'mesh': <transforms_templates.items.items.MeshItem at 0x7fb23da918d0>,
 'image2d': array([[[255., 255., 255., 255.],
         [255., 255., 255., 255.],
         [255., 255., 255., 255.],
         ...,
         [255., 255., 255., 255.],
         [255., 255., 255., 255.],
         [255., 255., 255., 255.]],
 
        [[255., 255., 255., 255.],
         [255., 255., 255., 255.],
         [255., 255., 255., 255.],
         ...,
         [255., 255., 255., 255.],
         [255., 255., 255., 255.],
         [255., 255., 255., 255.]],
 
        [[255., 255., 255., 255.],
         [255., 255., 255., 255.],
         [255., 255., 255., 255.],
         ...,
         [255., 255., 255., 255.],
         [255., 255., 255., 255.],
         [255., 255., 255., 255.]],
 
        ...,
 
        [[255., 255., 255., 255.],
         [255., 255., 255., 255.],
         [255., 255., 255., 255.],
         ...,
         [255., 255., 255., 255.],
         [255., 255., 255., 255.],
         [255., 255., 255., 255.

In [16]:
o['mesh'].data

<trimesh.Trimesh(vertices.shape=(20540, 3), faces.shape=(41068, 3))>

In [17]:
o['mesh'].data.metadata['ply_raw']['vertex']['data']['shape_id']

array([5, 3, 0, ..., 3, 5, 2], dtype=int8)

In [18]:
o['mesh'].data.metadata['ply_raw']['face']['data']['shape_id']

array([0, 0, 0, ..., 1, 1, 1], dtype=int8)

### Utils

In [21]:
from transforms_templates.transforms.utils import mesh_keys

In [22]:
mesh_keys(o)

['mesh']